# 📚 KM 文件管理工作流程實作

本教學將帶你實作完整的文件管理工作流程，包含：
1. 讀取文件清單（Manifest）
2. 上傳新文件
3. AI 條款切分處理
4. 關鍵字與實體查詢（模擬 GraphRAG）
5. 版本管理

## 資料夾結構
```
data/
├── manifest.json          # 📋 文件清單（主索引）
├── raw/                   # 📂 原始檔案
└── processed/             # 🤖 AI 處理後的 chunks
```

---

## 1. 環境設定與匯入

首先，我們需要匯入必要的 Python 套件，並設定資料夾路徑：

- **json**: 用於讀取和寫入 JSON 檔案
- **Path**: 用於處理檔案路徑，比字串更安全
- **datetime**: 用於記錄時間戳記
- **defaultdict**: 用於建立索引時的預設值字典

我們會定義四個重要的路徑變數：
- `DATA_DIR`: 資料根目錄
- `RAW_DIR`: 原始檔案目錄
- `PROCESSED_DIR`: AI 處理後的檔案目錄
- `MANIFEST_PATH`: 文件清單檔案路徑

In [ ]:
import json
import os
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Optional
from collections import defaultdict

# 設定資料路徑
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
MANIFEST_PATH = DATA_DIR / "manifest.json"

print(f"📁 資料目錄: {DATA_DIR.absolute()}")
print(f"📂 原始檔案: {RAW_DIR.absolute()}")
print(f"🤖 處理後檔案: {PROCESSED_DIR.absolute()}")
print(f"📋 清單檔案: {MANIFEST_PATH.absolute()}")

---

## 2. 文件清單管理類別 (ManifestManager)

這個類別負責管理 `manifest.json` 文件清單，提供以下功能：

| 方法 | 說明 |
|------|------|
| `list_documents()` | 列出所有文件 |
| `get_document(doc_id)` | 根據 ID 取得文件 |
| `add_document(doc_info)` | 新增文件 |
| `update_document(doc_id, updates)` | 更新文件資訊 |
| `search_by_category(category)` | 依分類搜尋 |
| `search_by_keyword(keyword)` | 依關鍵字搜尋 |

### 為什麼需要這個類別？

`manifest.json` 是整個系統的「目錄」，記錄了所有文件的基本資訊、版本歷史、處理狀態等。
透過這個類別，我們可以方便地操作這個清單，而不需要每次都手動讀寫 JSON 檔案。

In [ ]:
class ManifestManager:
    """文件清單管理器"""
    
    def __init__(self, manifest_path: Path):
        self.manifest_path = manifest_path
        self.manifest = self._load_manifest()
    
    def _load_manifest(self) -> dict:
        """載入清單"""
        if self.manifest_path.exists():
            with open(self.manifest_path, 'r', encoding='utf-8') as f:
                return json.load(f)
        return {"version": "1.0", "documents": []}
    
    def _save_manifest(self):
        """儲存清單"""
        self.manifest["updated_at"] = datetime.now().isoformat()
        with open(self.manifest_path, 'w', encoding='utf-8') as f:
            json.dump(self.manifest, f, ensure_ascii=False, indent=2)
    
    def list_documents(self) -> List[dict]:
        """列出所有文件"""
        return self.manifest.get("documents", [])
    
    def get_document(self, doc_id: str) -> Optional[dict]:
        """根據 ID 取得文件"""
        for doc in self.manifest["documents"]:
            if doc["doc_id"] == doc_id:
                return doc
        return None
    
    def add_document(self, doc_info: dict):
        """新增文件"""
        self.manifest["documents"].append(doc_info)
        self._save_manifest()
        print(f"✅ 已新增文件: {doc_info['doc_id']} - {doc_info['title']}")
    
    def update_document(self, doc_id: str, updates: dict):
        """更新文件資訊"""
        for doc in self.manifest["documents"]:
            if doc["doc_id"] == doc_id:
                doc.update(updates)
                self._save_manifest()
                print(f"✅ 已更新文件: {doc_id}")
                return
        print(f"❌ 找不到文件: {doc_id}")
    
    def search_by_category(self, category: str) -> List[dict]:
        """依分類搜尋"""
        return [doc for doc in self.manifest["documents"] 
                if doc.get("category") == category]
    
    def search_by_keyword(self, keyword: str) -> List[dict]:
        """依關鍵字搜尋"""
        results = []
        for doc in self.manifest["documents"]:
            keywords = doc.get("metadata", {}).get("keywords", [])
            if keyword in keywords or keyword in doc.get("title", ""):
                results.append(doc)
        return results

# 建立管理器實例
manifest_mgr = ManifestManager(MANIFEST_PATH)
print("✅ 文件清單管理器已初始化")

---

## 3. 檢視文件清單

現在讓我們來看看目前系統中有哪些文件。

這段程式會：
1. 呼叫 `list_documents()` 取得所有文件
2. 遍歷每份文件，顯示基本資訊
3. 顯示每份文件的最新版本資訊

你會看到類似這樣的輸出：
```
📄 LAW-001: 勞動基準法
   分類: 法規 > 勞動法規
   版本: v1
   狀態: active
   AI處理: ✅
   Chunks: 15
```

In [ ]:
# 列出所有文件
documents = manifest_mgr.list_documents()

print(f"📚 共有 {len(documents)} 份文件\n")
print("-" * 80)

for doc in documents:
    print(f"📄 {doc['doc_id']}: {doc['title']}")
    print(f"   分類: {doc['category']} > {doc.get('sub_category', 'N/A')}")
    print(f"   版本: v{doc['current_version']}")
    print(f"   狀態: {doc['status']}")
    print(f"   關鍵字: {', '.join(doc.get('metadata', {}).get('keywords', []))}")
    
    # 顯示版本歷史
    versions = doc.get('versions', [])
    if versions:
        latest = versions[-1]
        print(f"   最新版本:")
        print(f"      - 檔案: {latest['file_name']}")
        print(f"      - AI處理: {'✅' if latest.get('ai_processed') else '❌'}")
        print(f"      - Chunks: {latest.get('chunk_count', 'N/A')}")
    print("-" * 80)

---

## 4. Chunks 管理類別 (ChunksManager)

這是整個系統的核心！這個類別負責管理 AI 處理後的 chunks，支援 **GraphRAG 風格的查詢**。

### 什麼是 Chunk？

Chunk 是文件切分後的最小單位。在我們的系統中，每個「條款」就是一個 chunk。
例如勞基法第30條就是一個 chunk，包含：
- 條款原文
- 摘要
- 關鍵字（用於搜尋）
- 實體（角色、數值、程序等）
- 相關條款（用於 Graph 關聯）

### 這個類別建立了三種索引：

| 索引類型 | 說明 | 用途 |
|----------|------|------|
| `keyword_index` | 關鍵字 → chunk_ids | 關鍵字搜尋 |
| `entity_index` | 實體 → chunk_ids | 實體搜尋 |
| `article_index` | 條款編號 → chunk | 直接查詢特定條款 |

這些索引讓我們可以快速找到相關的條款，不需要遍歷所有資料。

In [ ]:
class ChunksManager:
    """條款 Chunks 管理器 - 支援 GraphRAG 查詢"""
    
    def __init__(self, processed_dir: Path):
        self.processed_dir = processed_dir
        self.chunks_cache = {}
        self._build_index()
    
    def _build_index(self):
        """建立索引 - 這是 GraphRAG 的基礎！"""
        self.keyword_index = defaultdict(list)  # keyword -> [chunk_ids]
        self.entity_index = defaultdict(list)   # entity -> [chunk_ids]
        self.article_index = {}                 # article_number -> chunk
        
        # 遍歷所有 _chunks.json 檔案
        for json_file in self.processed_dir.glob("*_chunks.json"):
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                doc_id = data['doc_id']
                self.chunks_cache[doc_id] = data
                
                # 建立每個 chunk 的索引
                for chunk in data.get('chunks', []):
                    chunk_id = chunk['chunk_id']
                    
                    # 關鍵字索引：讓我們可以用「加班」找到第32條
                    for kw in chunk.get('keywords', []):
                        self.keyword_index[kw].append(chunk_id)
                    
                    # 實體索引：讓我們可以用「雇主」找到相關條款
                    for entity in chunk.get('entities', []):
                        self.entity_index[entity['value']].append(chunk_id)
                    
                    # 條款索引：讓我們可以直接查詢「第30條」
                    self.article_index[chunk['article_number']] = chunk
        
        print(f"✅ 索引建立完成")
        print(f"   - 關鍵字: {len(self.keyword_index)} 個")
        print(f"   - 實體: {len(self.entity_index)} 個")
        print(f"   - 條款: {len(self.article_index)} 條")
    
    def search_by_keyword(self, keyword: str) -> List[dict]:
        """依關鍵字搜尋 chunks"""
        results = []
        for kw, chunk_ids in self.keyword_index.items():
            if keyword in kw:  # 部分匹配
                for chunk_id in chunk_ids:
                    chunk = self._get_chunk_by_id(chunk_id)
                    if chunk and chunk not in results:
                        results.append(chunk)
        return results
    
    def search_by_entity(self, entity_value: str) -> List[dict]:
        """依實體搜尋 chunks"""
        results = []
        for entity, chunk_ids in self.entity_index.items():
            if entity_value in entity:  # 部分匹配
                for chunk_id in chunk_ids:
                    chunk = self._get_chunk_by_id(chunk_id)
                    if chunk and chunk not in results:
                        results.append(chunk)
        return results
    
    def get_article(self, article_number: str) -> Optional[dict]:
        """取得特定條款"""
        return self.article_index.get(article_number)
    
    def get_related_articles(self, article_number: str) -> List[dict]:
        """取得相關條款 - 這就是 Graph 查詢！"""
        chunk = self.get_article(article_number)
        if not chunk:
            return []
        
        # 根據 related_articles 欄位找到關聯的條款
        related = []
        for rel_article in chunk.get('related_articles', []):
            rel_chunk = self.get_article(rel_article)
            if rel_chunk:
                related.append(rel_chunk)
        return related
    
    def _get_chunk_by_id(self, chunk_id: str) -> Optional[dict]:
        """根據 chunk_id 取得 chunk"""
        doc_id = chunk_id.rsplit('-', 1)[0]
        if doc_id in self.chunks_cache:
            for chunk in self.chunks_cache[doc_id].get('chunks', []):
                if chunk['chunk_id'] == chunk_id:
                    return chunk
        return None
    
    def get_all_keywords(self) -> List[str]:
        """取得所有關鍵字"""
        return list(self.keyword_index.keys())
    
    def get_all_entities(self) -> List[str]:
        """取得所有實體"""
        return list(self.entity_index.keys())

# 建立 Chunks 管理器
chunks_mgr = ChunksManager(PROCESSED_DIR)

---

## 5. 查看可用的關鍵字

在進行搜尋之前，讓我們先看看系統中有哪些關鍵字可供搜尋。

這些關鍵字是在 AI 處理階段從條款內容中抽取出來的，可以用來快速定位相關條款。

In [ ]:
# 顯示所有可用關鍵字
print("🏷️ 可用關鍵字：")
keywords = chunks_mgr.get_all_keywords()
print(", ".join(keywords[:20]) + "..." if len(keywords) > 20 else ", ".join(keywords))
print(f"\n總共 {len(keywords)} 個關鍵字")

---

## 6. 關鍵字搜尋示範：加班相關條款

現在來實際搜尋看看！我們用「加班」這個關鍵字來找相關條款。

這就是 GraphRAG 的第一步：**從關鍵字找到相關的 chunks**。

在實際的 RAG 系統中，這個步驟可能會結合向量搜尋（Embedding）來提高準確度。

In [ ]:
# 🔍 關鍵字搜尋：加班相關條款
print("🔍 搜尋關鍵字：加班\n")

results = chunks_mgr.search_by_keyword("加班")
print(f"找到 {len(results)} 條相關條款：\n")

for chunk in results:
    print(f"📌 {chunk['article_number']} - {chunk['title']}")
    print(f"   📝 摘要: {chunk['summary']}")
    print(f"   🏷️ 關鍵字: {', '.join(chunk['keywords'])}")
    print()

---

## 7. 關鍵字搜尋示範：特休相關條款

再試試搜尋「特休」，這是員工最關心的議題之一！

你會看到勞基法第38條的完整資訊，包括不同年資對應的特休天數。

In [ ]:
# 🔍 關鍵字搜尋：特休相關條款
print("🔍 搜尋關鍵字：特休\n")

results = chunks_mgr.search_by_keyword("特休")
for chunk in results:
    print(f"📌 {chunk['article_number']} - {chunk['title']}")
    print(f"   🏛️ 章節: {chunk['chapter']}")
    print(f"   📝 摘要: {chunk['summary']}")
    print(f"\n   📄 條文內容:")
    print(f"   {chunk['content'][:300]}...")
    print()

---

## 8. 查詢特定條款的完整資訊

有時候我們需要查看特定條款的完整資訊，包括：
- 條款原文
- 摘要
- 關鍵字
- 抽取的實體（角色、數值、程序等）
- 相關條款
- Metadata（條款類型、重要性、常見問題）

這些資訊在 GraphRAG 系統中會被用來：
1. 建立知識圖譜的節點和邊
2. 為 LLM 提供上下文
3. 生成更準確的回答

In [ ]:
# 📊 取得特定條款並顯示完整資訊
print("📊 查詢第30條 - 正常工作時間\n")
print("=" * 60)

article = chunks_mgr.get_article("第30條")
if article:
    print(f"📌 {article['article_number']} - {article['title']}")
    print(f"🏛️ 章節: {article['chapter']}")
    print(f"\n📝 條文內容:")
    print(f"   {article['content']}")
    print(f"\n📋 摘要: {article['summary']}")
    print(f"\n🏷️ 關鍵字: {', '.join(article['keywords'])}")
    print(f"\n🔗 相關條款: {', '.join(article['related_articles'])}")
    
    # 顯示實體
    print(f"\n🎯 抽取的實體:")
    for entity in article['entities']:
        print(f"   [{entity['type']}] {entity['value']}")
    
    # 顯示 metadata
    meta = article.get('metadata', {})
    print(f"\n📊 Metadata:")
    print(f"   類型: {meta.get('article_type', 'N/A')}")
    print(f"   重要性: {meta.get('importance', 'N/A')}")
    if 'common_questions' in meta:
        print(f"   常見問題: {', '.join(meta['common_questions'])}")
else:
    print("❌ 找不到該條款")

---

## 9. Graph 查詢：相關條款（GraphRAG 核心功能）

這是 **GraphRAG 最重要的功能**：根據條款之間的關聯進行查詢。

### 為什麼這很重要？

傳統的 RAG 系統只能根據關鍵字或向量相似度找到相關文件。
但 GraphRAG 可以理解文件之間的「關係」，例如：
- 第30條（正常工時）→ 第32條（延長工時/加班）
- 第30條（正常工時）→ 第36條（例假）

這樣當使用者問「正常工時是多少？加班怎麼算？」時，
系統可以自動找到相關的條款，提供更完整的答案。

In [ ]:
# 🔗 Graph 查詢：取得相關條款
print("🔗 Graph 查詢：與第30條相關的條款\n")
print("這個查詢模擬了 GraphRAG 的 Graph Traversal 功能")
print("=" * 60)

related = chunks_mgr.get_related_articles("第30條")
print(f"\n找到 {len(related)} 條相關條款:\n")

for chunk in related:
    print(f"📌 {chunk['article_number']} - {chunk['title']}")
    print(f"   📝 {chunk['summary']}")
    print(f"   🏷️ 關鍵字: {', '.join(chunk['keywords'][:5])}")
    print()

---

## 10. 查看所有實體

「實體」是從條款中抽取的重要概念，包括：
- **角色**：雇主、勞工
- **數值規定**：每日8小時、每週40小時
- **程序**：工會同意、勞資會議同意
- **機構**：基本工資審議委員會、行政院

這些實體在 GraphRAG 中會成為知識圖譜的節點。

In [ ]:
# 顯示所有實體
print("🎯 所有抽取的實體：\n")
entities = chunks_mgr.get_all_entities()

for i, entity in enumerate(entities[:20]):
    print(f"   {i+1:2d}. {entity}")

if len(entities) > 20:
    print(f"\n   ... 共 {len(entities)} 個實體")

---

## 11. 實體搜尋示範

現在用「實體」來搜尋相關條款。

例如搜尋「雇主」，可以找到所有提到雇主責任、義務的條款。
這對於回答「雇主需要負擔哪些責任？」這類問題非常有用。

In [ ]:
# 🔍 實體搜尋：雇主相關條款
print("🔍 搜尋實體：雇主\n")

results = chunks_mgr.search_by_entity("雇主")
print(f"找到 {len(results)} 條相關條款（顯示前5條）:\n")

for chunk in results[:5]:
    print(f"📌 {chunk['article_number']} - {chunk['title']}")
    print(f"   📝 {chunk['summary']}")
    print()

---

## 12. 模擬 AI 處理流程

這個函式模擬完整的 AI 處理流程，展示如何將原始文件轉換成 chunks。

### 實際應用中的 AI 處理步驟：

1. **條款切分**：使用 LLM 或規則將文件切成條款
2. **摘要生成**：使用 LLM 為每個條款生成摘要
3. **關鍵字抽取**：使用 NER 或 LLM 抽取關鍵字
4. **實體識別**：識別角色、數值、程序等實體
5. **關聯建立**：分析條款之間的引用關係

這個模擬版本會直接使用原始資料中的 keywords，
實際應用時會呼叫 OpenAI、Claude 等 LLM API。

In [ ]:
def simulate_ai_processing(raw_file_path: Path, output_path: Path) -> dict:
    """
    模擬 AI 處理流程
    
    實際應用中，這裡會呼叫 LLM API 進行：
    1. 條款切分
    2. 摘要生成
    3. 關鍵字抽取
    4. 實體識別
    5. 相關條款關聯
    """
    print(f"🤖 開始 AI 處理: {raw_file_path.name}")
    print("   1️⃣ 讀取原始檔案...")
    
    with open(raw_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print(f"   2️⃣ 解析條款... (共 {len(data.get('articles', []))} 條)")
    print("   3️⃣ 生成摘要...（模擬）")
    print("   4️⃣ 抽取關鍵字...")
    print("   5️⃣ 識別實體...（模擬）")
    print("   6️⃣ 建立條款關聯...（模擬）")
    
    # 模擬處理結果
    result = {
        "doc_id": "NEW-001",
        "title": data.get("law_name", "未知文件"),
        "version": 1,
        "processed_at": datetime.now().isoformat(),
        "total_chunks": len(data.get('articles', [])),
        "chunks": [],
        "ai_metadata": {
            "model": "gpt-4 (模擬)",
            "processing_time_ms": 5000
        }
    }
    
    # 轉換每個條款為 chunk
    for article in data.get('articles', []):
        chunk = {
            "chunk_id": f"NEW-001-{article['article_id'].replace('第', '').replace('條', '').zfill(3)}",
            "article_number": article['article_id'],
            "chapter": article.get('chapter', ''),
            "title": article.get('title', ''),
            "content": article.get('content', ''),
            "summary": f"（AI 生成摘要）{article.get('content', '')[:50]}...",
            "keywords": article.get('keywords', []),
            "entities": [],  # 實際應用中會由 AI 生成
            "related_articles": [],  # 實際應用中會由 AI 分析
            "metadata": {
                "importance": article.get('importance', 'medium')
            }
        }
        result["chunks"].append(chunk)
    
    # 儲存處理結果
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    
    print(f"\n✅ AI 處理完成！")
    print(f"   📁 輸出檔案: {output_path}")
    print(f"   📊 總 chunks: {result['total_chunks']}")
    
    return result

print("✅ AI 處理函式已定義")
print("\n💡 使用方式:")
print('   result = simulate_ai_processing(')
print('       RAW_DIR / "勞動基準法.json",')
print('       PROCESSED_DIR / "test_chunks.json"')
print('   )')

---

## 13. 新增文件流程示範

這個函式展示完整的新增文件流程：

1. **建立文件資訊** - 設定 doc_id、標題、分類等
2. **檢查原始檔案** - 確認檔案已放入 raw/ 資料夾
3. **AI 處理** - 呼叫 AI 進行條款切分
4. **更新清單** - 將文件資訊加入 manifest.json

這個函式目前只是展示流程，實際執行需要取消註解。

In [ ]:
def add_new_document(
    doc_id: str,
    title: str,
    category: str,
    sub_category: str,
    department: str,
    raw_file_name: str,
    keywords: List[str],
    uploaded_by: str = "admin"
) -> dict:
    """
    完整的新增文件流程
    """
    print(f"📄 新增文件: {doc_id} - {title}")
    print("=" * 50)
    
    # 1. 建立文件資訊
    doc_info = {
        "doc_id": doc_id,
        "title": title,
        "category": category,
        "sub_category": sub_category,
        "department": department,
        "current_version": 1,
        "status": "active",
        "versions": [
            {
                "version": 1,
                "file_name": raw_file_name,
                "raw_path": f"raw/{raw_file_name}",
                "processed_path": f"processed/{doc_id}_chunks.json",
                "uploaded_at": datetime.now().strftime("%Y-%m-%d"),
                "uploaded_by": uploaded_by,
                "description": "初版",
                "ai_processed": False,
                "chunk_count": 0
            }
        ],
        "metadata": {
            "keywords": keywords
        }
    }
    
    print(f"\n1️⃣ 建立文件資訊")
    print(f"   文件編號: {doc_id}")
    print(f"   分類: {category} > {sub_category}")
    
    # 2. 檢查原始檔案是否存在
    raw_path = RAW_DIR / raw_file_name
    print(f"\n2️⃣ 檢查原始檔案")
    if raw_path.exists():
        print(f"   ✅ 檔案存在: {raw_path}")
    else:
        print(f"   ⚠️ 檔案不存在: {raw_path}")
        print(f"   請先將檔案放入 raw/ 資料夾")
        return doc_info
    
    # 3. AI 處理（這裡只顯示說明，實際執行要取消註解）
    print(f"\n3️⃣ AI 處理")
    print(f"   ⏳ 待處理")
    # processed_path = PROCESSED_DIR / f"{doc_id}_chunks.json"
    # result = simulate_ai_processing(raw_path, processed_path)
    # doc_info['versions'][0]['ai_processed'] = True
    # doc_info['versions'][0]['chunk_count'] = result['total_chunks']
    
    # 4. 更新清單（這裡只顯示說明，實際執行要取消註解）
    print(f"\n4️⃣ 更新清單")
    print(f"   ⏳ 待更新")
    # manifest_mgr.add_document(doc_info)
    
    print(f"\n✅ 流程展示完成！")
    print(f"\n💡 實際執行時，請取消上面程式碼中的註解")
    return doc_info

print("✅ 新增文件函式已定義")
print("\n💡 使用範例:")
print('''   add_new_document(
       doc_id="HR-002",
       title="員工考勤管理辦法",
       category="人力資源",
       sub_category="考勤規定",
       department="人力資源部",
       raw_file_name="考勤管理辦法.json",
       keywords=["考勤", "打卡", "遲到", "出勤"]
   )''')

---

## 14. 統計資訊

最後，讓我們看看整個系統的統計資訊：
- 總文件數
- 分類統計
- Chunks 統計
- 關鍵字和實體數量

這些資訊可以幫助你了解知識庫的規模和覆蓋範圍。

In [ ]:
# 統計資訊
print("📊 系統統計")
print("=" * 50)

# 文件統計
documents = manifest_mgr.list_documents()
print(f"\n📚 文件統計:")
print(f"   總文件數: {len(documents)}")

# 分類統計
categories = defaultdict(int)
for doc in documents:
    categories[doc.get('category', '未分類')] += 1

print(f"\n🏷️ 分類統計:")
for cat, count in categories.items():
    print(f"   {cat}: {count} 份")

# Chunks 統計
total_chunks = sum(
    doc['versions'][-1].get('chunk_count', 0) 
    for doc in documents if doc.get('versions')
)

print(f"\n🧩 Chunks 統計:")
print(f"   總 chunks: {total_chunks}")
print(f"   關鍵字數: {len(chunks_mgr.get_all_keywords())}")
print(f"   實體數: {len(chunks_mgr.get_all_entities())}")

---

## 📝 總結

本教學涵蓋了完整的文件管理工作流程：

### 學到的內容

| 章節 | 內容 |
|------|------|
| 1-3 | Manifest 文件清單管理 |
| 4-5 | Chunks 管理與索引建立 |
| 6-7 | 關鍵字搜尋 |
| 8-9 | 條款查詢與 Graph 關聯 |
| 10-11 | 實體搜尋 |
| 12-13 | AI 處理與新增文件流程 |
| 14 | 統計資訊 |

### 下一步

- 🔌 整合實際的 LLM API（OpenAI、Claude）進行條款切分
- 📊 加入向量嵌入（Embedding）支援語意搜尋
- 🕸️ 建立完整的知識圖譜（Knowledge Graph）
- 💾 整合 MongoDB 儲存
- 🤖 結合 LangChain 或 LlamaIndex 建立完整的 RAG 系統